# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5380858b80>
├── 'a' --> tensor([[-0.5408,  1.2824, -3.0254],
│                   [-0.3561,  0.5099, -1.3856]])
└── 'x' --> <FastTreeValue 0x7f5380858c70>
    └── 'c' --> tensor([[-1.4677,  0.1053, -2.2673,  1.0475],
                        [-1.1240,  1.2008,  1.7960,  0.2016],
                        [ 0.4194, -1.5117,  0.3400, -2.8646]])

In [4]:
t.a

tensor([[-0.5408,  1.2824, -3.0254],
        [-0.3561,  0.5099, -1.3856]])

In [5]:
%timeit t.a

72 ns ± 3.32 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5380858b80>
├── 'a' --> tensor([[ 1.1376,  1.6752,  0.6943],
│                   [ 0.9414, -1.2118, -0.4436]])
└── 'x' --> <FastTreeValue 0x7f5380858c70>
    └── 'c' --> tensor([[-1.4677,  0.1053, -2.2673,  1.0475],
                        [-1.1240,  1.2008,  1.7960,  0.2016],
                        [ 0.4194, -1.5117,  0.3400, -2.8646]])

In [7]:
%timeit t.a = new_value

73.4 ns ± 1.82 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5408,  1.2824, -3.0254],
               [-0.3561,  0.5099, -1.3856]]),
    x: Batch(
           c: tensor([[-1.4677,  0.1053, -2.2673,  1.0475],
                      [-1.1240,  1.2008,  1.7960,  0.2016],
                      [ 0.4194, -1.5117,  0.3400, -2.8646]]),
       ),
)

In [10]:
b.a

tensor([[-0.5408,  1.2824, -3.0254],
        [-0.3561,  0.5099, -1.3856]])

In [11]:
%timeit b.a

77.8 ns ± 2.32 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0920, -0.2527, -1.4278],
               [-0.1100,  0.6860, -0.5857]]),
    x: Batch(
           c: tensor([[-1.4677,  0.1053, -2.2673,  1.0475],
                      [-1.1240,  1.2008,  1.7960,  0.2016],
                      [ 0.4194, -1.5117,  0.3400, -2.8646]]),
       ),
)

In [13]:
%timeit b.a = new_value

651 ns ± 19.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.04 µs ± 22.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.9 µs ± 905 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

310 µs ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

290 µs ± 8.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f52d6522610>
├── 'a' --> tensor([[[-0.5408,  1.2824, -3.0254],
│                    [-0.3561,  0.5099, -1.3856]],
│           
│                   [[-0.5408,  1.2824, -3.0254],
│                    [-0.3561,  0.5099, -1.3856]],
│           
│                   [[-0.5408,  1.2824, -3.0254],
│                    [-0.3561,  0.5099, -1.3856]],
│           
│                   [[-0.5408,  1.2824, -3.0254],
│                    [-0.3561,  0.5099, -1.3856]],
│           
│                   [[-0.5408,  1.2824, -3.0254],
│                    [-0.3561,  0.5099, -1.3856]],
│           
│                   [[-0.5408,  1.2824, -3.0254],
│                    [-0.3561,  0.5099, -1.3856]],
│           
│                   [[-0.5408,  1.2824, -3.0254],
│                    [-0.3561,  0.5099, -1.3856]],
│           
│                   [[-0.5408,  1.2824, -3.0254],
│                    [-0.3561,  0.5099, -1.3856]]])
└── 'x' --> <FastTreeValue 0x7f52dd15f1c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.1 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f52d658a4f0>
├── 'a' --> tensor([[-0.5408,  1.2824, -3.0254],
│                   [-0.3561,  0.5099, -1.3856],
│                   [-0.5408,  1.2824, -3.0254],
│                   [-0.3561,  0.5099, -1.3856],
│                   [-0.5408,  1.2824, -3.0254],
│                   [-0.3561,  0.5099, -1.3856],
│                   [-0.5408,  1.2824, -3.0254],
│                   [-0.3561,  0.5099, -1.3856],
│                   [-0.5408,  1.2824, -3.0254],
│                   [-0.3561,  0.5099, -1.3856],
│                   [-0.5408,  1.2824, -3.0254],
│                   [-0.3561,  0.5099, -1.3856],
│                   [-0.5408,  1.2824, -3.0254],
│                   [-0.3561,  0.5099, -1.3856],
│                   [-0.5408,  1.2824, -3.0254],
│                   [-0.3561,  0.5099, -1.3856]])
└── 'x' --> <FastTreeValue 0x7f52d658afa0>
    └── 'c' --> tensor([[-1.4677,  0.1053, -2.2673,  1.0475],
                        [-1.1240,  1.2008,  1.7960,  0.2016],
                 

In [23]:
%timeit t_cat(trees)

41.1 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.5 µs ± 2.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.5408,  1.2824, -3.0254],
                [-0.3561,  0.5099, -1.3856]],
       
               [[-0.5408,  1.2824, -3.0254],
                [-0.3561,  0.5099, -1.3856]],
       
               [[-0.5408,  1.2824, -3.0254],
                [-0.3561,  0.5099, -1.3856]],
       
               [[-0.5408,  1.2824, -3.0254],
                [-0.3561,  0.5099, -1.3856]],
       
               [[-0.5408,  1.2824, -3.0254],
                [-0.3561,  0.5099, -1.3856]],
       
               [[-0.5408,  1.2824, -3.0254],
                [-0.3561,  0.5099, -1.3856]],
       
               [[-0.5408,  1.2824, -3.0254],
                [-0.3561,  0.5099, -1.3856]],
       
               [[-0.5408,  1.2824, -3.0254],
                [-0.3561,  0.5099, -1.3856]]]),
    x: Batch(
           c: tensor([[[-1.4677,  0.1053, -2.2673,  1.0475],
                       [-1.1240,  1.2008,  1.7960,  0.2016],
                       [ 0.4194, -1.5117,  0.3400, -2.8646]],
         

In [26]:
%timeit Batch.stack(batches)

103 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.5408,  1.2824, -3.0254],
               [-0.3561,  0.5099, -1.3856],
               [-0.5408,  1.2824, -3.0254],
               [-0.3561,  0.5099, -1.3856],
               [-0.5408,  1.2824, -3.0254],
               [-0.3561,  0.5099, -1.3856],
               [-0.5408,  1.2824, -3.0254],
               [-0.3561,  0.5099, -1.3856],
               [-0.5408,  1.2824, -3.0254],
               [-0.3561,  0.5099, -1.3856],
               [-0.5408,  1.2824, -3.0254],
               [-0.3561,  0.5099, -1.3856],
               [-0.5408,  1.2824, -3.0254],
               [-0.3561,  0.5099, -1.3856],
               [-0.5408,  1.2824, -3.0254],
               [-0.3561,  0.5099, -1.3856]]),
    x: Batch(
           c: tensor([[-1.4677,  0.1053, -2.2673,  1.0475],
                      [-1.1240,  1.2008,  1.7960,  0.2016],
                      [ 0.4194, -1.5117,  0.3400, -2.8646],
                      [-1.4677,  0.1053, -2.2673,  1.0475],
                      [-1.1240,  

In [28]:
%timeit Batch.cat(batches)

185 µs ± 2.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

586 µs ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
